In [6]:
%pip install sshtunnel psycopg2-binary sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
from contextlib import closing


In [23]:
SSH_HOST = os.getenv('SSH_HOST')
SSH_PORT = int(os.getenv('SSH_PORT', 22))
SSH_USER = os.getenv('SSH_USER')
SSH_PASSWORD = os.getenv('SSH_PASSWORD')

DB_HOST = os.getenv('DB_HOST')
DB_PORT = int(os.getenv('DB_PORT', 5432))
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')


In [24]:
load_dotenv()

False

In [25]:
# # Create the SSH tunnel
# with SSHTunnelForwarder(
#     (SSH_HOST, SSH_PORT),
#     ssh_username=SSH_USER,
#     ssh_password=SSH_PASSWORD,
#     remote_bind_address=(DB_HOST, DB_PORT)) as tunnel:

#     conn = psycopg2.connect(
#         host='127.0.0.1',
#         port=tunnel.local_bind_port,
#         user=DB_USER,
#         password=DB_PASS,
#         database=DB_NAME)


In [26]:
# PGx gene list

# def fetch_prioritised_gene_regions(conn, gene_list):
#     query = f"""
#     SELECT chr, bp1, bp2, gene_name
#     FROM prada.gencode_gene
#     WHERE gene_name IN ({','.join(["%s"] * len(gene_list))})
#     """
#     return pd.read_sql(query, conn, params=gene_list)


In [ ]:
OUTPUT_BED = os.getenv("OUTPUT_BED", "pgx_genes.bed")


In [28]:
SQL = """
WITH sym_match AS (
  SELECT DISTINCT ON (pgx."Gene")
         pgx."Gene"                 AS pgx_gene,
         g.chr,
         g.bp1 - 1                  AS start0,
         g.bp2                      AS end1,
         g.gene_name,
         g.gene_id,
         'symbol'::text             AS match_rule
  FROM   prada.pgx_gene      pgx
  JOIN   prada.gencode_gene  g  ON LOWER(g.gene_name) = LOWER(pgx."Gene")
  WHERE  g.chr LIKE 'chr%'
  ORDER  BY pgx."Gene", g.bp1
), id_match AS (
  SELECT DISTINCT ON (pgx."Gene")
         pgx."Gene"                 AS pgx_gene,
         g.chr,
         g.bp1 - 1                  AS start0,
         g.bp2                      AS end1,
         g.gene_name,
         g.gene_id,
         'ensembl'::text            AS match_rule
  FROM   prada.pgx_gene      pgx
  JOIN   cpic.gene           c ON  c.symbol   = pgx."Gene"
  JOIN   prada.gencode_gene  g ON  g.gene_id  = c.ensemblid
  WHERE  pgx."Gene" NOT IN (SELECT pgx_gene FROM sym_match)
    AND  g.chr LIKE 'chr%'
  ORDER  BY pgx."Gene", g.bp1
), union_matches AS (
  SELECT * FROM sym_match
  UNION ALL
  SELECT * FROM id_match
)
SELECT
      chr,
      start0       AS chromStart,
      end1         AS chromEnd,
      pgx_gene     AS name
FROM   union_matches
ORDER  BY
  CASE
      WHEN chr = 'chrX'            THEN 23
      WHEN chr = 'chrY'            THEN 24
      WHEN chr IN ('chrM','chrMT') THEN 25
      WHEN chr ~ '^chr[0-9]+$'     THEN CAST(substr(chr,4) AS INT)
      ELSE 99
  END,
  start0;
"""


In [29]:
def fetch_bed_dataframe() -> pd.DataFrame:
    """Open SSH tunnel, run the query, return a DataFrame."""
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_pkey=SSH_PASSWORD,
        remote_bind_address=(DB_HOST, DB_PORT),
    ) as tunnel:

        local_port = tunnel.local_bind_port
        conn_str = f"dbname={DB_NAME} user={DB_USER} password={DB_PASS} host=127.0.0.1 port={local_port}"

        with closing(psycopg2.connect(conn_str)) as conn, conn.cursor() as cur:
            cur.execute(SQL)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]

    return pd.DataFrame(rows, columns=cols)

In [31]:
if __name__ == "__main__":
    df = fetch_bed_dataframe()
    df.to_csv(OUTPUT_BED, sep="\t", header=False, index=False)

    print(f"Wrote {len(df):,d} records to {OUTPUT_BED}")

AssertionError: IP is not a string (NoneType)